# Creating Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    
    #can it assign task to any other agent
    allow_delegation=False,
    
    #we can see what it is thinking
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Introduction:
Artificial Intelligence (AI) is revolutionizing industries and shaping our future in ways we never thought possible. In this blog article, we will delve into the latest trends, key players, and noteworthy news surrounding AI to provide valuable insights for our audience.

Key Points:
1. Latest Trends in AI:
- Machine Learning: The rapid advancements in machine learning algorithms are driving innovation 

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Artificial Intelligence (AI) continues to be at the forefront of technological innovation, shaping industries and revolutionizing the way we live and work. In this blog post, we delve into the latest trends, key players, and noteworthy news in the AI landscape to provide valuable insights to our audience.

## Latest Trends in AI

**Machine Learning** remains a key driver of AI advancements, with continuous improvements in algorithms leading to innovative applications in various sectors. From personalized recommendations to fraud detection, machine learning is paving the way for groundbreaking developments in AI technology.

**Natural Language Processing (NLP)** is another pivotal trend in AI, empowering machines to comprehend and generate human language. This has led to significant breakthroughs in chatbots and virtual assistants, reshaping how we int

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Artificial Intelligence (AI) continues to be at the forefront of technological innovation, shaping industries and revolutionizing the way we live and work. In this blog post, we delve into the latest trends, key players, and noteworthy news in the AI landscape to provide valuable insights to our audience.

## Latest Trends in AI

**Machine Learning** remains a key driver of AI advancements, with continuous improvements in algorithms leading to innovative applications in various sectors. From personalized recommendations to fraud detection, machine learning is paving the way for groundbreaking developments in AI technology.

**Natural Language Processing (NLP)** is another pivotal trend in AI, empowering machines to comprehend and generate human language. This has led to significant breakthroughs in chatbots and virtual assistants, reshaping how we interact with technology and opening up new possibilities for AI integration.

**AI Ethics** is a growing concern as AI becomes more prevalent in our daily lives. Discussions surrounding bias, privacy, and transparency are crucial as we navigate the ethical implications of AI development. Addressing these considerations is essential to ensure responsible and fair implementation of AI technologies.

**Autonomous Vehicles** have emerged as a prominent topic in the AI space, with major players like Tesla, Waymo, and Uber investing heavily in self-driving car technology. While the potential for autonomous vehicles to revolutionize transportation is vast, challenges related to safety and regulation must be carefully addressed for widespread adoption.

## Key Players in AI

**Google** stands out with its AI research division, DeepMind, making significant strides in AI research and development. From AlphaGo to language translation, Google's AI initiatives are pushing the boundaries of what AI can achieve and driving innovation across various domains.

**Amazon** has made a mark in the AI space with its AI-powered personal assistant, Alexa, leading the market in smart home devices. With a focus on natural language understanding and personalized recommendations, Alexa has become an indispensable part of many households, showcasing the potential of AI in enhancing daily experiences.

**IBM** is revolutionizing industries with its Watson AI platform, offering cognitive computing capabilities that help businesses make informed decisions and enhance efficiency. From healthcare to finance, Watson is empowering organizations to leverage AI-powered insights for transformative outcomes.

## Noteworthy News in AI

**OpenAI's GPT-3** has garnered attention for its ability to generate human-like text, sparking discussions on the future of AI-generated content. As AI advances, exploring the implications of AI-generated content on creativity and authenticity becomes crucial for understanding the evolving landscape of AI technology.

**Microsoft's Acquisition of Nuance** underscores the growing significance of AI in healthcare, with the integration of conversational AI technologies poised to revolutionize patient care and outcomes. Microsoft's strategic move highlights the transformative potential of AI in improving healthcare delivery and enhancing patient experiences.

## Conclusion

Artificial Intelligence is a dynamic force reshaping industries and redefining the way we interact with technology. By staying informed and engaged with the latest AI developments, we can harness the transformative power of AI while remaining mindful of ethical considerations. As AI continues to evolve, it is imperative for professionals and decision-makers to stay informed and proactive in navigating the evolving AI landscape.

**Call to Action**: Stay informed and engaged with the latest AI trends and developments by following reputable sources and connecting with industry leaders. Embrace the opportunities that AI offers while upholding ethical standards for responsible AI integration.

Our target audience, comprising tech enthusiasts, industry professionals, and decision-makers, can benefit from the insights provided on AI trends, key players, and noteworthy news in this blog post. By offering valuable perspectives on the evolving AI landscape, we aim to empower our audience to make informed decisions in the ever-evolving realm of AI technology.

Incorporating SEO keywords such as "Artificial Intelligence trends," "Key players in AI," "Latest news on AI," "Machine learning advancements," and "AI ethics implications" enables us to reach a wider audience interested in staying abreast of AI developments.

In conclusion, Artificial Intelligence holds immense potential to drive innovation and enhance our lives. By engaging with the latest AI trends and advancements, we can leverage the power of AI to create a brighter future for all. This blog post serves as a valuable resource for understanding the evolving AI landscape and its impact on various industries.

This revised blog post is now aligned with the brand's voice and ready for publication, providing comprehensive insights and engaging content on the future of Artificial Intelligence.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).